In [1]:
import pandas as pd
import math

In [2]:
dataset = pd.read_csv('final_datasets/all_health_appended.csv')
dataset.head()

,measure_id,measure_name,country,cause_id,cause_name,metric_id,metric_name,year,value,upper,lower,iso3
0,1,Deaths,Honduras,703,Poisoning by other means,1,Number,1990,61.099487,77.418662,47.345706,HND
1,1,Deaths,Honduras,703,Poisoning by other means,2,Percent,1990,0.002212,0.002781,0.001731,HND
2,1,Deaths,Honduras,703,Poisoning by other means,3,Rate,1990,1.297629,1.644216,1.005527,HND
3,1,Deaths,Bangladesh,521,Cirrhosis and other chronic liver diseases,1,Number,1990,26628.429130,32225.285311,20798.897909,BGD
4,1,Deaths,Bangladesh,521,Cirrhosis and other chronic liver diseases,2,Percent,1990,0.023659,0.028261,0.018677,BGD


In [3]:
dataset.shape

(2386752, 12)

In [4]:
dataset['measure_name'].unique()

array(['Deaths', 'YLLs (Years of Life Lost)',
       'DALYs (Disability-Adjusted Life Years)', 'Incidence',
       'YLDs (Years Lived with Disability)', 'Prevalence'], dtype=object)

In [5]:
filters = dataset[['cause_id', 'cause_name']]
filters = filters.drop_duplicates()
filters.head()

,cause_id,cause_name
0,703,Poisoning by other means
3,521,Cirrhosis and other chronic liver diseases
6,526,Digestive diseases
9,527,Peptic ulcer disease
12,528,Gastritis and duodenitis


In [6]:
filters.reset_index(drop=True, inplace=True)
filters.rename({'cause_id': 'value', 'cause_name': 'label'}, axis=1, inplace=True)

filters.shape

(98, 2)

### Add health codes, parent, and level

In [7]:
dictionary = pd.read_csv('data/health_codes.csv')
dictionary.head()

,Cause ID,Cause Name,Parent ID,Parent Name,Level
0,294,All causes,294,All causes,0
1,295,"Communicable, maternal, neonatal, and nutritio...",294,All causes,1
2,955,HIV/AIDS and sexually transmitted infections,295,"Communicable, maternal, neonatal, and nutritio...",2
3,298,HIV/AIDS,955,HIV/AIDS and sexually transmitted infections,3
4,948,HIV/AIDS - Drug-susceptible Tuberculosis,298,HIV/AIDS,4


In [8]:
dictionary.rename({'Cause ID': 'value', 'Cause Name': 'label', 'Parent ID': 'parent_id', 
                   'Parent Name': 'parent_name', 'Level': 'level'}, axis=1, inplace=True)
dictionary.head()

,value,label,parent_id,parent_name,level
0,294,All causes,294,All causes,0
1,295,"Communicable, maternal, neonatal, and nutritio...",294,All causes,1
2,955,HIV/AIDS and sexually transmitted infections,295,"Communicable, maternal, neonatal, and nutritio...",2
3,298,HIV/AIDS,955,HIV/AIDS and sexually transmitted infections,3
4,948,HIV/AIDS - Drug-susceptible Tuberculosis,298,HIV/AIDS,4


In [9]:
# USING NAMES 
# # get unique conditions
# unique_in_filters = filters['label'].unique()
# print('\nUnique unique_in_filters:', len(unique_in_filters))
# unique_in_dictionary = dictionary['label'].unique()
# print('Unique unique_in_dictionary:', len(unique_in_dictionary))

# # find the foods not in filters
# difference_1 = list(set(unique_in_filters) - set(unique_in_dictionary))
# print('\nfoods not in filters', len(difference_1), ':', difference_1)

# # find the foods not in dictionary
# difference_2 = list(set(unique_in_dictionary) - set(unique_in_filters))
# difference_2.sort()
# print('\nfoods not in dictionary', len(difference_2), ':', difference_2)


# USING CODES 
# get unique conditions
unique_in_filters = filters['value'].unique()
print('\nUnique unique_in_filters:', len(unique_in_filters))
unique_in_dictionary = dictionary['value'].unique()
print('Unique unique_in_dictionary:', len(unique_in_dictionary))

# find the diseases not in filters
difference_1 = list(set(unique_in_filters) - set(unique_in_dictionary))
print('\ndiseases not in filters', len(difference_1), ':', difference_1)

# find the diseases not in dictionary
difference_2 = list(set(unique_in_dictionary) - set(unique_in_filters))
difference_2.sort()
print('\ndiseases not in dictionary', len(difference_2), ':', difference_2)


Unique unique_in_filters: 98
Unique unique_in_dictionary: 360

foods not in filters 2 : [1028, 1029]

foods not in dictionary 264 : [294, 295, 297, 298, 300, 302, 319, 320, 321, 322, 328, 329, 332, 337, 338, 339, 340, 341, 342, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 367, 368, 369, 370, 374, 375, 376, 379, 381, 382, 383, 384, 385, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 408, 409, 411, 414, 418, 419, 420, 423, 441, 444, 447, 450, 459, 462, 490, 492, 493, 495, 496, 497, 500, 501, 502, 503, 504, 509, 510, 511, 512, 513, 514, 516, 520, 522, 523, 524, 525, 541, 547, 548, 557, 560, 561, 562, 563, 564, 565, 566, 569, 573, 574, 579, 582, 585, 588, 592, 595, 596, 597, 598, 602, 604, 605, 606, 607, 608, 609, 612, 614, 615, 616, 618, 619, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 656, 657, 658, 659, 660, 662, 663, 665, 668, 669, 670, 671, 672, 674, 675, 679, 680, 681, 6

# Do I want to add some of the conditions not present in the current list?

# parent_id == NaN if the category is parent level
    use value and label values, add level = 1
    for [1028, 1029] parent_id, parent_name and level are NaNs (not available in the dictionary dataset, and are top level)
    then typecast parent_id and level as int (not allowed as NaN values cannot be converted)


In [10]:
mapping = pd.merge(filters, dictionary, on=["value", "label"], how='left')
mapping

,value,label,parent_id,parent_name,level
0,703,Poisoning by other means,700.0,Poisonings,4.0
1,521,Cirrhosis and other chronic liver diseases,526.0,Digestive diseases,3.0
2,526,Digestive diseases,409.0,Non-communicable diseases,2.0
3,527,Peptic ulcer disease,992.0,Upper digestive system diseases,4.0
4,528,Gastritis and duodenitis,992.0,Upper digestive system diseases,4.0
5,529,Appendicitis,526.0,Digestive diseases,3.0
6,530,Paralytic ileus and intestinal obstruction,526.0,Digestive diseases,3.0
7,531,"Inguinal, femoral, and abdominal hernia",526.0,Digestive diseases,3.0
8,532,Inflammatory bowel disease,526.0,Digestive diseases,3.0
9,533,Vascular intestinal disorders,526.0,Digestive diseases,3.0


In [11]:
mapping.shape

(98, 5)

In [12]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [13]:
mapping.sort_values(by=["parent_name"], inplace=True)
mapping

,value,label,parent_id,parent_name,level
34,687,Injuries,294.0,All causes,1.0
25,499,Cardiomyopathy and myocarditis,491.0,Cardiovascular diseases,3.0
24,498,Hypertensive heart disease,491.0,Cardiovascular diseases,3.0
23,494,Stroke,491.0,Cardiovascular diseases,3.0
41,997,Chronic kidney disease due to diabetes mellitus type 1,589.0,Chronic kidney disease,4.0
42,998,Chronic kidney disease due to diabetes mellitus type 2,589.0,Chronic kidney disease,4.0
20,593,Chronic kidney disease due to other and unspecified causes,589.0,Chronic kidney disease,4.0
19,591,Chronic kidney disease due to hypertension,589.0,Chronic kidney disease,4.0
52,515,Asthma,508.0,Chronic respiratory diseases,3.0
27,971,Cirrhosis and other chronic liver diseases due to NAFLD,521.0,Cirrhosis and other chronic liver diseases,4.0


In [14]:
for index, row in mapping.iterrows():
#     if row['value'] == 1029:
    if math.isnan(row['parent_id']):
        mapping.at[index, 'parent_name']= row['label']
        mapping.at[index, 'parent_id']= row['value']
        mapping.at[index, 'level']= 999

In [15]:
mapping

,value,label,parent_id,parent_name,level
34,687,Injuries,294.0,All causes,1.0
25,499,Cardiomyopathy and myocarditis,491.0,Cardiovascular diseases,3.0
24,498,Hypertensive heart disease,491.0,Cardiovascular diseases,3.0
23,494,Stroke,491.0,Cardiovascular diseases,3.0
41,997,Chronic kidney disease due to diabetes mellitus type 1,589.0,Chronic kidney disease,4.0
42,998,Chronic kidney disease due to diabetes mellitus type 2,589.0,Chronic kidney disease,4.0
20,593,Chronic kidney disease due to other and unspecified causes,589.0,Chronic kidney disease,4.0
19,591,Chronic kidney disease due to hypertension,589.0,Chronic kidney disease,4.0
52,515,Asthma,508.0,Chronic respiratory diseases,3.0
27,971,Cirrhosis and other chronic liver diseases due to NAFLD,521.0,Cirrhosis and other chronic liver diseases,4.0


In [16]:
mapping.dtypes

value          int64  
label          object 
parent_id      float64
parent_name    object 
level          float64
dtype: object

In [17]:
mapping['parent_id'] = mapping['parent_id'].astype(int)
mapping['level'] = mapping['level'].astype(int)

In [18]:
mapping.dtypes

value          int64 
label          object
parent_id      int64 
parent_name    object
level          int64 
dtype: object

In [19]:
mapping['label'].unique()

array(['Injuries', 'Cardiomyopathy and myocarditis',
       'Hypertensive heart disease', 'Stroke',
       'Chronic kidney disease due to diabetes mellitus type 1',
       'Chronic kidney disease due to diabetes mellitus type 2',
       'Chronic kidney disease due to other and unspecified causes',
       'Chronic kidney disease due to hypertension', 'Asthma',
       'Cirrhosis and other chronic liver diseases due to NAFLD',
       'Nutritional deficiencies', 'Maternal and neonatal disorders',
       'Major depressive disorder', 'Seborrhoeic dermatitis',
       'Atopic dermatitis', 'Chronic kidney disease', 'Diabetes mellitus',
       'Diabetes mellitus type 2', 'Diabetes mellitus type 1',
       'Pancreatitis', 'Gallbladder and biliary diseases',
       'Vascular intestinal disorders', 'Inflammatory bowel disease',
       'Inguinal, femoral, and abdominal hernia',
       'Paralytic ileus and intestinal obstruction', 'Appendicitis',
       'Cirrhosis and other chronic liver diseases',
 

In [20]:
excluded_list = ['Injuries', 'Unintentional injuries', 'Self-harm and interpersonal violence', 
                 'Transport injuries', 'Poisoning by other means', 'Interpersonal violence', 
                 'Maternal and neonatal disorders', 'Maternal disorders', 'Neonatal disorders', 
                 'Low back pain', 'Chronic respiratory diseases', 'Self-harm', 
                 'Total burden related to Non-alcoholic fatty liver disease (NAFLD)', 'Total cancers']



In [21]:
curated_conditions = mapping

for condition in excluded_list:
    curated_conditions.drop(mapping[mapping['label'] == condition].index, inplace = True)
    print('Removing', condition, '...')

Removing Injuries ...
Removing Unintentional injuries ...
Removing Self-harm and interpersonal violence ...
Removing Transport injuries ...
Removing Poisoning by other means ...
Removing Interpersonal violence ...
Removing Maternal and neonatal disorders ...
Removing Maternal disorders ...
Removing Neonatal disorders ...
Removing Low back pain ...
Removing Chronic respiratory diseases ...
Removing Self-harm ...
Removing Total burden related to Non-alcoholic fatty liver disease (NAFLD) ...
Removing Total cancers ...


In [22]:
print('Injuries' in curated_conditions['label'].values)

False


In [23]:
curated_conditions

curated_conditions.sort_values(by=['label'])

,value,label,parent_id,parent_name,level
97,661,Acne vulgaris,653,Skin and subcutaneous diseases,3
47,543,Alzheimer's disease and other dementias,542,Neurological disorders,3
79,571,Anxiety disorders,558,Mental disorders,3
5,529,Appendicitis,526,Digestive diseases,3
52,515,Asthma,508,Chronic respiratory diseases,3
83,977,Atopic dermatitis,654,Dermatitis,4
81,578,Attention-deficit/hyperactivity disorder,558,Mental disorders,3
80,575,Autism spectrum disorders,558,Mental disorders,3
78,570,Bipolar disorder,558,Mental disorders,3
64,474,Bladder cancer,410,Neoplasms,3


In [24]:
# save all conditions to json
# filters.to_json('final_datasets/health_filters.json', orient="records")

In [25]:
# save curated conditions to json
curated_conditions.to_json('final_datasets/health_filters_curated.json', orient="records")

In [26]:
# save merged dataset 
# mapping.to_json('final_datasets/merged_health_filters.json', orient="records")